Inspired by sentdex's video series at: https://www.youtube.com/watch?v=PwAGxqrXSCs

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
#Load MNIST
mnist = input_data.read_data_sets('/tmp/data/',one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [85]:
n_features = 28**2
n_classes = 10
n_nodes = [n_features,200,n_classes]
batch_size = 100

#Define placeholders
x = tf.placeholder('float',[None,n_features]) #28x28 images
y = tf.placeholder('float')

def neural_network_model(data,n_nodes):
    #Construct a neural network model with ReLU activation functions of with depth of hidden layers lenght(n_nodes) 
    # where n_nodes details the number of node per layer. (n_nodes[0]=input,...,n_nodes[-1]=output)
    n_layers = len(n_nodes)
    #Initialize weights vectors
    weights = []
    biases = []
    #Initialize input
    layer = data
    for l in range(n_layers):
        curr_nodes = n_nodes[l]
        next_nodes = n_nodes[l+1]
        
        #Initialize weights for current layer
        weights.append(tf.Variable(tf.random_normal([curr_nodes,next_nodes])))
        biases.append(tf.Variable(tf.random_normal([next_nodes])))
        
        #Create connections
        # Next layer = activation function(previous layer * weights + biases)
        if l == n_layers-2:
            #Output
            output = tf.matmul(layer,weights[l]) + biases[l]
            return output
        else:
            layer = tf.nn.relu(tf.matmul(layer,weights[l]) + biases[l])

def train_neural_network(x,epochs=10):
    #Train nn
    #Output from nn
    prediction = neural_network_model(x,n_nodes)
    #Cost of prediction
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y) )
    #Backprop
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #Run session
    with tf.Session() as sess:
        """TRAIN"""
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            loss = 0
            #For each batch
            for _ in range(int(mnist.train.num_examples/batch_size)):
                #Data and labels
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimizer,cost],feed_dict={x:epoch_x,y:epoch_y})
                loss += c
            print('Epoch: %d out of %d with loss %.3f'%(epoch,epochs,loss))
        
        """TEST"""
        #Calculate accuracy   
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: %.3f'%accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

In [86]:
train_neural_network(x)

Tensor("add_77:0", shape=(?, 10), dtype=float32)
Epoch: 0 out of 10 with loss 11575.941
Epoch: 1 out of 10 with loss 2851.577
Epoch: 2 out of 10 with loss 1875.074
Epoch: 3 out of 10 with loss 1396.404
Epoch: 4 out of 10 with loss 1087.098
Epoch: 5 out of 10 with loss 870.263
Epoch: 6 out of 10 with loss 709.925
Epoch: 7 out of 10 with loss 587.784
Epoch: 8 out of 10 with loss 482.822
Epoch: 9 out of 10 with loss 408.197
Accuracy: 0.935
